# Grandes cidades brasileiras são progressistas?

In [ ]:
import pandas as pd
import six
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import ListedColormap
import seaborn as sns
import itertools
import geopandas as gpd

from pandas.plotting import table 
from IPython.display import HTML
from unidecode import unidecode

%matplotlib inline

# Ler repositório de votação do TSE (1996-2018)

Dados compilados pelo Brasil.io: https://brasil.io/dataset/eleicoes-brasil/votacoes/

In [ ]:
cols = ['ano_eleicao', 'descricao_totalizacao_turno', 
        'codigo_municipio', 'nome_municipio', 'sigla_uf', 'nome_partido', 'numero_partido', 
        'sigla_partido', 'descricao_cargo', 'nome', 'numero_urna', 'numero_zona', 'total_votos']

df_chunk = pd.read_csv('input/2019-04-08/votacao-zona.csv', sep=',', usecols=cols, encoding='latin1', chunksize=1000000)

In [ ]:
chunk_list = []

for chunk in df_chunk:  
    chunk_filter = chunk[chunk.descricao_cargo == 'VEREADOR']
    chunk_filter = chunk_filter[chunk_filter.descricao_totalizacao_turno.isin(['ELEITO', 'ELEITO POR QP', 'ELEITO POR MEDIA'])]
    chunk_filter.drop(['descricao_totalizacao_turno', 'descricao_cargo'], axis=1, inplace=True)
    chunk_list.append(chunk_filter)    
df_concat = pd.concat(chunk_list)

# Dados parciais 2020

In [ ]:
cols = ['codtse','ncand','nome','partido','votabs','situacao','uf']
df_2020 = pd.read_csv('input/vereadores2020.csv', sep=';', usecols=cols)

In [ ]:
df_2020 = df_2020[df_2020.situacao == 'Eleito']
df_2020.drop('situacao', axis=1, inplace=True)
df_2020.columns = ['codigo_municipio', 'numero_urna', 'nome', 'sigla_partido', 'total_votos', 'sigla_uf']

Criar colunas para satisfazer dataset principal

In [ ]:
df_2020['ano_eleicao'] = 2020
df_2020['numero_partido'] = df_2020['numero_urna'].astype(str).str[:2].astype(int)
df_2020['numero_zona'] = 0

In [ ]:
df_temp = pd.read_excel('input/tse_partidos.xlsx')
dict_partidos = pd.Series(df_temp.NOME.values, index = df_temp.LEGENDA.values).to_dict()
df_2020['nome_partido'] = df_2020.numero_partido.map(dict_partidos)

In [ ]:
dict_sigla_partidos = pd.Series(df_temp.SIGLA.values, index = df_temp.LEGENDA.values).to_dict()
df_2020['sigla_partido'] = df_2020.numero_partido.map(dict_sigla_partidos)

In [ ]:
df_temp = df_concat[['codigo_municipio', 'nome_municipio']].drop_duplicates('codigo_municipio', keep='first').sort_values('codigo_municipio')
dict_municipios = pd.Series(df_temp.nome_municipio.values, index = df_temp.codigo_municipio.values).to_dict()
df_2020['nome_municipio'] = df_2020.codigo_municipio.map(dict_municipios)

Agrupar zonas eleitorais

In [ ]:
df = df_concat.append(df_2020, sort=False)

In [ ]:
df = df.groupby(['ano_eleicao','codigo_municipio','numero_urna',
                'nome_municipio','sigla_uf','nome_partido','numero_partido','sigla_partido'])['total_votos'].sum().reset_index()

# Filtrar por ano da eleição

In [ ]:
ano_da_eleicao = df.ano_eleicao.unique().tolist()[1:]

Renomear partidos antigos

In [ ]:
df.sigla_partido.replace('PRN','PTC', inplace=True)
df.sigla_partido.replace('PPB','PP', inplace=True)
df.sigla_partido.replace('PFL','DEM', inplace=True)
df.sigla_partido.replace('PTN','PODE', inplace=True)
df.sigla_partido.replace('PT DO B','AVT', inplace=True)
df.sigla_partido.replace('PEN','PATRI', inplace=True)
df.sigla_partido.replace('PMDB','MDB', inplace=True)
df.sigla_partido.replace('PSDC','DC', inplace=True)
df.sigla_partido.replace('PPS','CDD', inplace=True)
df.sigla_partido.replace('PR','PL', inplace=True)

# Mapa ideológico dos partidos

In [ ]:
mapa_ideologico = {'PCB':'Esquerda',
                    'PT':'Esquerda',
                    'UP':'Esquerda',
                    'PSOL':'Esquerda',
                    'PSTU':'Esquerda',
                    'PC DO B':'Esquerda',
                    'PCO':'Esquerda',
                    'PDT':'Esquerda',
                    'REDE':'Esquerda',
                    'PSB':'Esquerda',
                    'PV':'Esquerda',
                    'SD':'Centro',
                    'PROS':'Centro',
                    'PTB':'Centro',
                    'PRB':'Centro',
                    'PP':'Centro',
                    'PMN':'Centro',
                    'PMB':'Centro',
                    'MDB':'Centro',
                    'PSDB':'Centro',
                    'CDD':'Centro',
                    'PSD':'Centro',
                    'DEM':'Direita',
                    'PL':'Direita',
                    'DC':'Direita',
                    'AVT':'Direita',
                    'PSC':'Direita',
                    'PODE':'Direita',
                    'PATRI':'Direita',
                    'NOVO':'Direita',
                    'PRTB':'Direita',
                    'PSL':'Direita',
                    'PTC':'Direita',
                    'PGT':'Esquerda',
                    'PST':'Esquerda',
                    'PHS':'Direita',
                    'PRP':'Direita',
                    'PPL':'Esquerda',
                    'PAN':'Centro',
                    'PRONA':'Direita'}

In [ ]:
df['ideologia'] = df.sigla_partido.map(mapa_ideologico)

# Lista de população, tamanho e densidade por município

In [ ]:
sigla_estado = {'AC':'Acre',
                'AL':'Alagoas',
                'AP':'Amapa',
                'AM':'Amazonas',
                'BA':'Bahia',
                'CE':'Ceara',
                'DF':'Distrito Federal',
                'ES':'Espirito Santos',
                'GO':'Goias',
                'MA':'Maranhao',
                'MT':'Mato Grosso',
                'MS':'Mato Grosso do Sul',
                'MG':'Minas Gerais',
                'PA':'Para',
                'PB':'Paraiba',
                'PR':'Parana',
                'PE':'Pernambuco',
                'PI':'Piaui',
                'RJ':'Rio de Janeiro',
                'RN':'Rio Grande do Norte',
                'RS':'Rio Grande do Sul',
                'RO':'Rondonia',
                'RR':'Roraima',
                'SC':'Santa Catarina',
                'SP':'Sao Paulo',
                'SE':'Sergipe',
                'TO':'Tocantins'}

In [ ]:
regioes = {'Norte':['AC','AP','AM','PA','RO','RR','TO'],
           'Nordente':['AL','BA','CE','MA','PB','PE','PI','RN','SE'],
           'Centro-Oeste':['GO','MT','MS'],
           'Sudeste':['ES','MG','RJ','SP'],
           'Sul':['PR','SC','RN']}

In [ ]:
capitais = {'AC':'Rio Branco',
            'AL':'Maceio',
            'AP':'Macapa',
            'AM':'Manaus',
            'BA':'Salvador',
            'CE':'Fortaleza',
            'DF':'Brasilia',
            'ES':'Vitoria',
            'GO':'Goiania',
            'MA':'São Luis',
            'MT':'Cuiaba',
            'MS':'Campo Grande',
            'MG':'Belo Horizonte',
            'PA':'Belem',
            'PB':'Joao Pessoa',
            'PR':'Curitiba',
            'PE':'Recife',
            'PI':'Teresina',
            'RJ':'Rio de Janeiro',
            'RN':'Natal',
            'RS':'Porto Alegre',
            'RO':'Porto Velho',
            'RR':'Boa Vista',
            'SC':'Florianopolis',
            'SP':'Sao Paulo',
            'SE':'Aracaju',
            'TO':'Palmas'}

Obter lista da população, tamanho e densidade por município

In [ ]:
url = 'https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_do_Brasil_por_popula%C3%A7%C3%A3o_(2020)'
municipio_pop = pd.read_html(url, thousands=' ')[0]

# eliminar espaço para converter para int
municipio_pop['População'] = municipio_pop['População'].str.replace('\D', '').astype(int)

# converter estado e municipio para sem acentos e maiusculo
municipio_pop['Unidade federativa'] = municipio_pop['Unidade federativa'].str.upper().apply(lambda x: unidecode(x))
municipio_pop['Município'] = municipio_pop['Município'].str.upper().apply(lambda x: unidecode(x))

# abreviar UF
municipio_pop['sigla_uf'] = municipio_pop['Unidade federativa'].str.upper().apply(lambda x: [k for k, v in sigla_estado.items() if (unidecode(v.upper()) == x)])
municipio_pop = municipio_pop.explode('sigla_uf')

municipio_pop.drop('Posição', axis=1, inplace=True)
municipio_pop.set_index('Código IBGE', inplace=True)

In [ ]:
url = 'https://pt.m.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_brasileiros_por_%C3%A1rea_decrescente'
municipio_km = pd.read_html(url, thousands=' ')[0]

municipio_km['Área (km²)'] = municipio_km['Área (km²)'].str.replace(',','.').str.replace(u'\xa0', u'').astype(float)
municipio_km.drop(['Posição','Município', 'Unidade federativa'], axis=1, inplace=True)
municipio_km.rename(columns={'Código do IBGE':'Código IBGE'}, inplace=True)
municipio_km.set_index('Código IBGE', inplace=True)

In [ ]:
density_label = ['0-5','5-10','10-20','20-50','50-100','100-200','200-20k']

def grupo_densidade(den):
    if den >= 0 and den < 5:
        return 0
    elif den >= 5 and den < 10:
        return 1
    elif den >= 10 and den < 20:
        return 2
    elif den >= 20 and den < 50:
        return 3
    elif den >= 50 and den < 100:
        return 4
    elif den >= 100 and den < 200:
        return 5
    elif den >= 200 and den < 20000:
        return 6
    else:
        return 'other'

In [ ]:
municipios = pd.merge(municipio_pop, municipio_km, on='Código IBGE')
municipios['densidade'] = municipios['População'] / municipios['Área (km²)']

In [ ]:
municipios['grupo_densidade'] = municipios.apply(lambda x : grupo_densidade(x.densidade), axis=1)

In [ ]:
# Obter tabela de correspondencia entre codigo de municipio do IBGE e TSE
link = 'https://raw.githubusercontent.com/estadao/como-votou-sua-vizinhanca/master/data/votos/correspondencia-tse-ibge.csv'
df_equivalencia = pd.read_csv(link)
df_equivalencia.set_index('COD_TSE', drop=False, inplace=True)
df_equivalencia.drop(['chave','NOME','UF'], axis=1, inplace=True)

In [ ]:
df.set_index('codigo_municipio', drop=False, inplace=True)
df = df.join(df_equivalencia)

In [ ]:
df.set_index('GEOCOD_IBGE', drop=False, inplace=True)
df = df.join(municipios[['População','densidade','grupo_densidade']])

Latitude e Longitude

In [ ]:
# Obter tabela latitude e longitude para municipios
link = 'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv'
df_lat_long = pd.read_csv(link)
df_lat_long.set_index('codigo_ibge', inplace=True)
df_lat_long.drop(['nome','capital','codigo_uf'], axis=1, inplace=True)

In [ ]:
df = df.join(df_lat_long)

# Funções auxiliares

In [ ]:
def group_by_density(df, ideology=False):
    res = pd.DataFrame()
    for i, val in enumerate(ano_da_eleicao):
        # Filtrar pelo ano e remover municipios duplicados
        df_by_year = df[df.ano_eleicao == val]

        # Agrupar municipios eleitos por partido
        df_temp = df_by_year.groupby('sigla_partido').count().sort_values('codigo_municipio', ascending=False).iloc[:,0]
        df_temp = df_temp.reset_index(name=val).set_index('sigla_partido')
       
        res = res.join(df_temp, how='outer') 
        
    res.fillna(0,inplace=True)    
    return res

In [ ]:
def group_by_year(df):
    res = pd.DataFrame()
    for i, val in enumerate(ano_da_eleicao):
        # Filtrar pelo ano e remover municipios duplicados
        df_by_year = df[df.ano_eleicao == val]

        # Agrupar municipios eleitos por partido
        df_temp = df_by_year.groupby('sigla_partido').count().sort_values('codigo_municipio', ascending=False).iloc[:,0]
        df_temp = df_temp.reset_index(name=val).set_index('sigla_partido')
       
        res = res.join(df_temp, how='outer') 
        
    res.fillna(0,inplace=True)    
    return res

In [ ]:
def draw_barchart(year, df, ax, title, textstr):
    res_by_year = df[[year]].dropna().sort_values(by=year, ascending=False).astype(int).head(10)
    ax.clear()
    b = sns.barplot(y=res_by_year.index, x=res_by_year.iloc[:,0], data=res_by_year, palette='plasma', ax=ax)

    plt.title(title, fontsize=25, fontweight='bold')
    plt.xlabel('Municípios', fontsize=15, fontweight='bold')
    plt.ylabel('Partidos', fontsize=15, fontweight='bold')

    b.set_yticks([])
    plt.xticks(fontsize=15)

    dx = res_by_year.iloc[:,0].max() / 100
    pos = range(res_by_year.shape[0])
    for tick in pos:
        ax.text(res_by_year.iloc[tick,0]-dx, pos[tick], res_by_year.index[tick], ha='right', va='center',
                fontsize=25, fontweight='bold', color='w')
        ax.text(res_by_year.iloc[tick,0]+dx, pos[tick], res_by_year.iloc[tick,0], ha='left', va='center',
                fontsize=20, fontweight='bold')
        
    ax.text(0.8, 0.4, str(year), transform=ax.transAxes, color='#777777', size=60, ha='right', weight=800)
        
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax.text(0.8, 0.1, textstr, transform=ax.transAxes, fontsize=15,
            verticalalignment='top', bbox=props)

In [ ]:
def max_size(column_lenght, max_size=5):
    if column_lenght > max_size:
        size = max_size
    else:
        size = column_lenght
    return size

def draw_subplots(axs, r_esq, r_centro, r_dir, labels=5):
    l_esq = sns.lineplot(data=r_esq, dashes=False, legend=False, linewidth=5, marker='o',
                     markersize=15, palette='Reds_r', ax=axs[0])
    l_centro = sns.lineplot(data=r_centro, dashes=False, legend=False, linewidth=5, marker='o',
                 markersize=15, palette='YlOrBr_r', ax=axs[1])
    l_dir = sns.lineplot(data=r_dir, dashes=False, legend=False, linewidth=5, marker='o',
                     markersize=15, palette='Blues_r', ax=axs[2])

    axs[0].set_title('Esquerda', loc='left', fontsize=20, fontweight='bold', color='grey')
    axs[1].set_title('Centro', loc='left', fontsize=20, fontweight='bold', color='grey')
    axs[2].set_title('Direita', loc='left', fontsize=20, fontweight='bold', color='grey')  
    
    axs[0].set_ylabel('Municípios', labelpad=2, fontsize=15, fontweight='bold', color='grey')
    axs[1].set_ylabel('Municípios', labelpad=2, fontsize=15, fontweight='bold', color='grey')
    axs[2].set_ylabel('Municípios', labelpad=2, fontsize=15, fontweight='bold', color='grey')
  
    axs[0].set_xticks(np.arange(ano_da_eleicao[0], ano_da_eleicao[-1]+2, step=4))
    axs[1].set_xticks(np.arange(ano_da_eleicao[0], ano_da_eleicao[-1]+2, step=4))
    axs[2].set_xticks(np.arange(ano_da_eleicao[0], ano_da_eleicao[-1]+2, step=4))
    
    axs[0].set_xticklabels(labels=[int(i) for i in axs[0].get_xticks()], fontsize=15, color='grey')
    axs[1].set_xticklabels(labels=[int(i) for i in axs[1].get_xticks()], fontsize=15, color='grey')
    axs[2].set_xticklabels(labels=[int(i) for i in axs[2].get_xticks()], fontsize=15, color='grey')
    
    axs[0].set_yticklabels(labels=[int(i) for i in axs[0].get_yticks()], fontsize=15, color='grey')
    axs[1].set_yticklabels(labels=[int(i) for i in axs[1].get_yticks()], fontsize=15, color='grey')
    axs[2].set_yticklabels(labels=[int(i) for i in axs[2].get_yticks()], fontsize=15, color='grey')
    
    axs[0].spines['bottom'].set_color('grey')
    axs[0].spines['left'].set_color('grey')
    axs[1].spines['bottom'].set_color('grey')
    axs[1].spines['left'].set_color('grey') 
    axs[2].spines['bottom'].set_color('grey')
    axs[2].spines['left'].set_color('grey')     

    for i in range(max_size(len(axs[0].get_lines()),labels)):
        axs[0].text(r_esq.index[-1]+.2, r_esq.iloc[-1,i], r_esq.columns[i],
                    fontsize=15, fontweight='bold', color=axs[0].get_lines()[i].get_color())

    for i in range(max_size(len(axs[0].get_lines()),5)):
        axs[1].text(r_centro.index[-1]+.2, r_centro.iloc[-1,i], r_centro.columns[i],
                    fontsize=15, fontweight='bold', color=axs[1].get_lines()[i].get_color())

    for i in range(max_size(len(axs[0].get_lines()),4)):
        axs[2].text(r_dir.index[-1]+.2, r_dir.iloc[-1,i], r_dir.columns[i],
                    fontsize=15, fontweight='bold', color=axs[2].get_lines()[i].get_color())
        
    axs[0].axvline(2005, color='grey', linestyle='--')
    axs[0].axvline(2013, color='grey', linestyle='--')
    axs[0].axvline(2014, color='grey', linestyle='--')
    axs[0].axvline(2016, color='grey', linestyle='--')

    axs[1].axvline(2005, color='grey', linestyle='--')
    axs[1].axvline(2013, color='grey', linestyle='--')
    axs[1].axvline(2014, color='grey', linestyle='--')
    axs[1].axvline(2016, color='grey', linestyle='--')
    
    axs[2].axvline(2005, color='grey', linestyle='--')
    axs[2].axvline(2013, color='grey', linestyle='--')
    axs[2].axvline(2014, color='grey', linestyle='--')
    axs[2].axvline(2016, color='grey', linestyle='--')

    axs[0].text(0.275, 0.99, 'Mensalão', transform=axs[0].transAxes, rotation=-90,
                fontsize=15, color='dimgrey', verticalalignment='top')
    axs[0].text(0.635, 0.99, 'Manifestações', transform=axs[0].transAxes, rotation=-90,
                fontsize=15, color='dimgrey', verticalalignment='top')
    axs[0].text(0.685, 0.99, 'Lava Jato', transform=axs[0].transAxes, rotation=-90,
                fontsize=15, color='dimgrey', verticalalignment='top')
    axs[0].text(0.775, 0.99, 'Impeachment', transform=axs[0].transAxes, rotation=-90,
                fontsize=15, color='dimgrey', verticalalignment='top')

# Evolução de vereadores eleitos por partido

### Todos Municípios

In [ ]:
res = pd.DataFrame()
res = group_by_year(df)

Preparar dataset para flourish

In [ ]:
res.insert(0, 'ideologia', res.index.map(mapa_ideologico))
res.to_excel('output/flourish_vereadores_municipios_total.xlsx')

Agrupar por ideologia dos partidos


In [ ]:
r_esq = res[res.ideologia == 'Esquerda']
r_esq = r_esq.drop('ideologia', axis=1).T
r_esq = r_esq.fillna(0).astype(int)
r_esq.sort_values(r_esq.index[-1], axis=1, ascending=False, inplace=True)

r_centro = res[res.ideologia == 'Centro']
r_centro = r_centro.drop('ideologia', axis=1).T
r_centro = r_centro.fillna(0).astype(int)
r_centro.sort_values(r_centro.index[-1], axis=1, ascending=False, inplace=True)

r_dir = res[res.ideologia == 'Direita']
r_dir = r_dir.drop('ideologia', axis=1).T
r_dir = r_dir.fillna(0).astype(int)
r_dir.sort_values(r_dir.index[-1], axis=1, ascending=False, inplace=True)

In [ ]:
fig, axs = plt.subplots(3,1, figsize = (20,20))

sns.set_style('white')
sns.despine()

title = 'Vereadores Eleitos por Partido (todos municípios)'
legend = '\n'.join((
    r'Fonte: TSE, 2020',
    r'Autor: Guilherme Chinellato'
))

plt.text(x=0.125, y=0.9, s=title, fontsize=25, color='grey', fontweight='bold',
         transform=fig.transFigure)

axs[0].text(1.0, 1.0, legend, transform=axs[0].transAxes, fontsize=12, ha='right', va='top', color='grey')

draw_subplots(axs, r_esq, r_centro, r_dir) 

plt.savefig('output/vereadores_partido_total.png', dpi=300, bbox_inches = 'tight')

Resultado agregado

In [ ]:
agg_total = pd.DataFrame(columns=['Esquerda','Centro','Direita'], index=ano_da_eleicao)
for k,v in enumerate(r_esq.index):
    agg_total.loc[v,'Esquerda'] = r_esq.loc[v].sum()

for k,v in enumerate(r_centro.index):
    agg_total.loc[v,'Centro'] = r_centro.loc[v].sum()
    
for k,v in enumerate(r_dir.index):
    agg_total.loc[v,'Direita'] = r_dir.loc[v].sum()

agg_total_var = agg_total.pct_change().stack().reset_index()
agg_total_var.columns = ['Ano','Ideologia', 'Variação']
    
agg_total

### Capitais

In [ ]:
# converter estado e municipio para sem acentos e maiusculo
capitais_upper = {k.upper() : unidecode(v.upper()) for k,v in capitais.items() }
del capitais_upper['DF']

# eliminar municipios que possuem o mesmo nome, mas são de diferentes estados
df_temp = df.assign(nome_sigla_municipio = df.sigla_uf + ',' + df.nome_municipio)

# somente municipios das capitais
lista_capitais = [ k + ',' + v for k,v in capitais_upper.items() ]
df_capitais = df_temp[(df_temp.nome_sigla_municipio.isin(lista_capitais))]

In [ ]:
res_capitais = pd.DataFrame()
res_capitais = group_by_year(df_capitais)

Preparar dataset para flourish

In [ ]:
res_capitais.insert(0, 'ideologia', res_capitais.index.map(mapa_ideologico))
res_capitais.to_excel('output/flourish_vereadores_municipios_capitais.xlsx')

Agrupar por ideologia dos partidos

In [ ]:
r_esq_capitais = res_capitais[res_capitais.ideologia == 'Esquerda']
r_esq_capitais = r_esq_capitais.drop('ideologia', axis=1).T
r_esq_capitais = r_esq_capitais.fillna(0).astype(int)
r_esq_capitais.sort_values(r_esq_capitais.index[-1], axis=1, ascending=False, inplace=True)

r_centro_capitais = res_capitais[res_capitais.ideologia == 'Centro']
r_centro_capitais = r_centro_capitais.drop('ideologia', axis=1).T
r_centro_capitais = r_centro_capitais.fillna(0).astype(int)
r_centro_capitais.sort_values(r_centro_capitais.index[-1], axis=1, ascending=False, inplace=True)

r_dir_capitais = res_capitais[res_capitais.ideologia == 'Direita']
r_dir_capitais = r_dir_capitais.drop('ideologia', axis=1).T
r_dir_capitais = r_dir_capitais.fillna(0).astype(int)
r_dir_capitais.sort_values(r_dir_capitais.index[-1], axis=1, ascending=False, inplace=True)

Resultado agregado

In [ ]:
agg_capitais = pd.DataFrame(columns=['Esquerda','Centro','Direita'], index=ano_da_eleicao)
for k,v in enumerate(r_esq_capitais.index):
    agg_capitais.loc[v,'Esquerda'] = r_esq_capitais.loc[v].sum()
    
for k,v in enumerate(r_centro_capitais.index):
    agg_capitais.loc[v,'Centro'] = r_centro_capitais.loc[v].sum()
    
for k,v in enumerate(r_dir_capitais.index):
    agg_capitais.loc[v,'Direita'] = r_dir_capitais.loc[v].sum()
    
agg_capitais_var = agg_capitais.pct_change().stack().reset_index()
agg_capitais_var.columns = ['Ano','Ideologia', 'Variação']

agg_capitais

### Cidades Grandes: >500mil hab.

In [ ]:
res_municipios_grandes = pd.DataFrame()
res_municipios_grandes = group_by_year(df[df['População'] >= 500000])

In [ ]:
res_municipios_grandes.insert(0, 'ideologia', res_municipios_grandes.index.map(mapa_ideologico))
res_municipios_grandes.to_excel('output/flourish_vereadores_municipios_grandes.xlsx')

Agrupar por ideologia dos partidos

In [ ]:
r_esq_municipios_grandes = res_municipios_grandes[res_municipios_grandes.ideologia == 'Esquerda']
r_esq_municipios_grandes = r_esq_municipios_grandes.drop('ideologia', axis=1).T
r_esq_municipios_grandes = r_esq_municipios_grandes.fillna(0).astype(int)
r_esq_municipios_grandes.sort_values(r_esq_municipios_grandes.index[-1], axis=1, ascending=False, inplace=True)

r_centro_municipios_grandes = res_municipios_grandes[res_municipios_grandes.ideologia == 'Centro']
r_centro_municipios_grandes = r_centro_municipios_grandes.drop('ideologia', axis=1).T
r_centro_municipios_grandes = r_centro_municipios_grandes.fillna(0).astype(int)
r_centro_municipios_grandes.sort_values(r_centro_municipios_grandes.index[-1], axis=1, ascending=False, inplace=True)

r_dir_municipios_grandes = res_municipios_grandes[res_municipios_grandes.ideologia == 'Direita']
r_dir_municipios_grandes = r_dir_municipios_grandes.drop('ideologia', axis=1).T
r_dir_municipios_grandes = r_dir_municipios_grandes.fillna(0).astype(int)
r_dir_municipios_grandes.sort_values(r_dir_municipios_grandes.index[-1], axis=1, ascending=False, inplace=True)

Resultado agregado

In [ ]:
agg_municipios_grandes = pd.DataFrame(columns=['Esquerda','Centro','Direita'], index=ano_da_eleicao)
for k,v in enumerate(r_esq_municipios_grandes.index):
    agg_municipios_grandes.loc[v,'Esquerda'] = r_esq_municipios_grandes.loc[v].sum()

for k,v in enumerate(r_centro_municipios_grandes.index):
    agg_municipios_grandes.loc[v,'Centro'] = r_centro_municipios_grandes.loc[v].sum()
    
for k,v in enumerate(r_dir_municipios_grandes.index):
    agg_municipios_grandes.loc[v,'Direita'] = r_dir_municipios_grandes.loc[v].sum()

agg_municipios_grandes_var = agg_municipios_grandes.pct_change().stack().reset_index()
agg_municipios_grandes_var.columns = ['Ano','Ideologia', 'Variação']
    
agg_municipios_grandes

### Cidades Médias: >100mil e <500mil hab.

In [ ]:
res_municipios_medios = pd.DataFrame()
res_municipios_medios = group_by_year(df[(df['População'] >= 100000) & (df['População'] < 500000)])

In [ ]:
res_municipios_medios.insert(0, 'ideologia', res_municipios_medios.index.map(mapa_ideologico))
res_municipios_medios.to_excel('output/flourish_vereadores_municipios_medios.xlsx')

Agrupar por ideologia dos partidos

In [ ]:
r_esq_municipios_medios = res_municipios_medios[res_municipios_medios.ideologia == 'Esquerda']
r_esq_municipios_medios = r_esq_municipios_medios.drop('ideologia', axis=1).T
r_esq_municipios_medios = r_esq_municipios_medios.fillna(0).astype(int)
r_esq_municipios_medios.sort_values(r_esq_municipios_medios.index[-1], axis=1, ascending=False, inplace=True)

r_centro_municipios_medios = res_municipios_medios[res_municipios_medios.ideologia == 'Centro']
r_centro_municipios_medios = r_centro_municipios_medios.drop('ideologia', axis=1).T
r_centro_municipios_medios = r_centro_municipios_medios.fillna(0).astype(int)
r_centro_municipios_medios.sort_values(r_centro_municipios_medios.index[-1], axis=1, ascending=False, inplace=True)

r_dir_municipios_medios = res_municipios_medios[res_municipios_medios.ideologia == 'Direita']
r_dir_municipios_medios = r_dir_municipios_medios.drop('ideologia', axis=1).T
r_dir_municipios_medios = r_dir_municipios_medios.fillna(0).astype(int)
r_dir_municipios_medios.sort_values(r_dir_municipios_medios.index[-1], axis=1, ascending=False, inplace=True)

Resultado agregado

In [ ]:
agg_municipios_medios = pd.DataFrame(columns=['Esquerda','Centro','Direita'], index=ano_da_eleicao)
for k,v in enumerate(r_esq_municipios_medios.index):
    agg_municipios_medios.loc[v,'Esquerda'] = r_esq_municipios_medios.loc[v].sum()

for k,v in enumerate(r_centro_municipios_medios.index):
    agg_municipios_medios.loc[v,'Centro'] = r_centro_municipios_medios.loc[v].sum()
    
for k,v in enumerate(r_dir_municipios_medios.index):
    agg_municipios_medios.loc[v,'Direita'] = r_dir_municipios_medios.loc[v].sum()

agg_municipios_medios_var = agg_municipios_medios.pct_change().stack().reset_index()
agg_municipios_medios_var.columns = ['Ano','Ideologia', 'Variação']
    
agg_municipios_medios

### Cidades Pequenas: <100mil hab.

In [ ]:
res_municipios_pequenos = pd.DataFrame()
res_municipios_pequenos = group_by_year(df[df['População'] < 100000])

In [ ]:
res_municipios_pequenos.insert(0, 'ideologia', res_municipios_pequenos.index.map(mapa_ideologico))
res_municipios_pequenos.to_excel('output/flourish_vereadores_municipios_pequenos.xlsx')

Agrupar por ideologia dos partidos

In [ ]:
r_esq_municipios_pequenos = res_municipios_pequenos[res_municipios_pequenos.ideologia == 'Esquerda']
r_esq_municipios_pequenos = r_esq_municipios_pequenos.drop('ideologia', axis=1).T
r_esq_municipios_pequenos = r_esq_municipios_pequenos.fillna(0).astype(int)
r_esq_municipios_pequenos.sort_values(r_esq_municipios_pequenos.index[-1], axis=1, ascending=False, inplace=True)

r_centro_municipios_pequenos = res_municipios_pequenos[res_municipios_pequenos.ideologia == 'Centro']
r_centro_municipios_pequenos = r_centro_municipios_pequenos.drop('ideologia', axis=1).T
r_centro_municipios_pequenos = r_centro_municipios_pequenos.fillna(0).astype(int)
r_centro_municipios_pequenos.sort_values(r_centro_municipios_pequenos.index[-1], axis=1, ascending=False, inplace=True)

r_dir_municipios_pequenos = res_municipios_pequenos[res_municipios_pequenos.ideologia == 'Direita']
r_dir_municipios_pequenos = r_dir_municipios_pequenos.drop('ideologia', axis=1).T
r_dir_municipios_pequenos = r_dir_municipios_pequenos.fillna(0).astype(int)
r_dir_municipios_pequenos.sort_values(r_dir_municipios_pequenos.index[-1], axis=1, ascending=False, inplace=True)

Resultado agregado

In [ ]:
agg_municipios_pequenos = pd.DataFrame(columns=['Esquerda','Centro','Direita'], index=ano_da_eleicao)
for k,v in enumerate(r_esq_municipios_pequenos.index):
    agg_municipios_pequenos.loc[v,'Esquerda'] = r_esq_municipios_pequenos.loc[v].sum()

for k,v in enumerate(r_centro_municipios_pequenos.index):
    agg_municipios_pequenos.loc[v,'Centro'] = r_centro_municipios_pequenos.loc[v].sum()
    
for k,v in enumerate(r_dir_municipios_pequenos.index):
    agg_municipios_pequenos.loc[v,'Direita'] = r_dir_municipios_pequenos.loc[v].sum()

agg_municipios_pequenos_var = agg_municipios_pequenos.pct_change().stack().reset_index()
agg_municipios_pequenos_var.columns = ['Ano','Ideologia', 'Variação']
    
agg_municipios_pequenos

### Densidade Populacional

In [ ]:
res_den = pd.DataFrame()
for i, val in enumerate(ano_da_eleicao):
    df_temp = df[df.ano_eleicao == val].drop_duplicates(subset='codigo_municipio', keep='first')
    df_temp = df_temp.groupby(['grupo_densidade','ideologia'])['ideologia'].count().unstack()
    df_temp.fillna(0, inplace=True)
    df_temp['year'] = val
    res_den = res_den.append(df_temp) 
    
res_den.reset_index(drop=False, inplace=True)
res_den = res_den[['year','grupo_densidade','Esquerda','Centro','Direita']]

res_den.to_excel('output/flourish_vereadores_municipios_densidade.xlsx')

In [ ]:
res_den_maior = res_den[res_den.grupo_densidade == 6].copy()
res_den_maior.drop('grupo_densidade',axis=1,inplace=True)
res_den_maior.set_index('year', inplace=True)

res_den_menor = res_den[res_den.grupo_densidade != 6].groupby('year').sum()
res_den_menor.drop('grupo_densidade',axis=1,inplace=True)

res_den_maior_var = res_den_maior.pct_change().stack().reset_index()
res_den_maior_var.columns = ['Ano','Ideologia','Variação']

res_den_menor_var = res_den_menor.pct_change().stack().reset_index()
res_den_menor_var.columns = ['Ano','Ideologia','Variação']

# Evolução de prefeitos eleitos por ideologia do partido

In [ ]:
col = ['Ano','Município','Esquerda','Centro','Direita']

r = pd.DataFrame()
for i, val in enumerate(ano_da_eleicao):    
    t = pd.DataFrame(columns=col)

    t['Ano'] = [val,val,val]
    t['Município'] = ['Grandes','Médios','Pequenos']

    t.iloc[0,2] = round((agg_municipios_grandes.loc[val,'Esquerda'] / agg_municipios_grandes.loc[val].sum())*100,2)
    t.iloc[0,3] = round((agg_municipios_grandes.loc[val,'Centro'] / agg_municipios_grandes.loc[val].sum())*100,2)
    t.iloc[0,4] = round((agg_municipios_grandes.loc[val,'Direita'] / agg_municipios_grandes.loc[val].sum())*100,2)

    t.iloc[1,2] = round((agg_municipios_medios.loc[val,'Esquerda'] / agg_municipios_medios.loc[val].sum())*100,2)
    t.iloc[1,3] = round((agg_municipios_medios.loc[val,'Centro'] / agg_municipios_medios.loc[val].sum())*100,2)
    t.iloc[1,4] = round((agg_municipios_medios.loc[val,'Direita'] / agg_municipios_medios.loc[val].sum())*100,2)

    t.iloc[2,2] = round((agg_municipios_pequenos.loc[val,'Esquerda'] / agg_municipios_pequenos.loc[val].sum())*100,2)
    t.iloc[2,3] = round((agg_municipios_pequenos.loc[val,'Centro'] / agg_municipios_pequenos.loc[val].sum())*100,2)
    t.iloc[2,4] =round((agg_municipios_pequenos.loc[val,'Direita'] / agg_municipios_pequenos.loc[val].sum())*100,2)
    
    r = r.append(t)
    
r.to_excel('output/flourish_vereadores_ideologia_perc.xlsx')
r

In [ ]:
colors = ['red', 'orange', 'dodgerblue']

fig, axs = plt.subplots(3,1, figsize = (20,15))

sns.set_style('white')
sns.despine()

title='Vereadores Eleitos por Ideologia do Partido'
plt.text(x=0.125, y=0.91, s=title, fontsize=25, color='grey', fontweight='bold',
         transform=fig.transFigure)

l_grande = sns.lineplot(data=agg_municipios_grandes, dashes=False, legend=False, linewidth=5.5,
                       palette=colors, marker='o', markersize=15, ax=axs[0])
l_medio = sns.lineplot(data=agg_municipios_medios, dashes=False, legend=False, linewidth=5.5,
                       palette=colors, marker='o', markersize=15, ax=axs[1])
l_pequeno = sns.lineplot(data=agg_municipios_pequenos, dashes=False, legend=False, linewidth=5.5,
                       palette=colors, marker='o', markersize=15, ax=axs[2])

axs[0].set_title('Municípios Grandes', loc='left',
                 fontsize=20, fontweight='bold', color='grey')
axs[1].set_title('Municípios Médios', loc='left',
                 fontsize=20, fontweight='bold', color='grey')
axs[2].set_title('Municípios Pequenos', loc='left',
                 fontsize=20, fontweight='bold', color='grey')

axs[0].set_xlabel('')
axs[1].set_xlabel('')
axs[2].set_xlabel('')

axs[0].set_ylabel('Municípios', labelpad=2, fontsize=15, fontweight='bold', color='grey')
axs[1].set_ylabel('Municípios', labelpad=2, fontsize=15, fontweight='bold', color='grey')
axs[2].set_ylabel('Municípios', labelpad=2, fontsize=15, fontweight='bold', color='grey')

axs[0].set_xticks(np.arange(ano_da_eleicao[0], ano_da_eleicao[-1]+2, step=4))
axs[1].set_xticks(np.arange(ano_da_eleicao[0], ano_da_eleicao[-1]+2, step=4))
axs[2].set_xticks(np.arange(ano_da_eleicao[0], ano_da_eleicao[-1]+2, step=4))

axs[0].set_xticklabels(labels=[int(i) for i in axs[0].get_xticks()], fontsize=15, color='grey')
axs[1].set_xticklabels(labels=[int(i) for i in axs[1].get_xticks()], fontsize=15, color='grey')
axs[2].set_xticklabels(labels=[int(i) for i in axs[2].get_xticks()], fontsize=15, color='grey')

axs[0].set_yticklabels(labels=[int(i) for i in axs[0].get_yticks()], fontsize=15, color='grey')
axs[1].set_yticklabels(labels=[int(i) for i in axs[1].get_yticks()], fontsize=15, color='grey')
axs[2].set_yticklabels(labels=[int(i) for i in axs[2].get_yticks()], fontsize=15, color='grey')

axs[0].spines['bottom'].set_color('grey')
axs[0].spines['left'].set_color('grey')
axs[1].spines['bottom'].set_color('grey')
axs[1].spines['left'].set_color('grey') 
axs[2].spines['bottom'].set_color('grey')
axs[2].spines['left'].set_color('grey') 

#def flip(items, ncol):
#    return itertools.chain(*[items[i::ncol] for i in range(ncol)])

#handles, labels = axs[0].get_legend_handles_labels()
#leg = l_grande.legend(flip(handles, 4), flip(labels, 4), loc=2, ncol=4, fontsize=15, borderaxespad=0.)    
#for line in leg.get_lines():
#    line.set_linewidth(5.5)

axs[0].text(1.0, 1.1, legend, transform=axs[0].transAxes, fontsize=12, ha='right', va='top', color='grey')

for i in range(len(axs[0].get_lines())):
    axs[0].text(agg_municipios_grandes.index[-1]+.2, agg_municipios_grandes.iloc[-1,i], agg_municipios_grandes.columns[i],
                fontsize=15, fontweight='bold', color=axs[0].get_lines()[i].get_color())

for i in range(len(axs[1].get_lines())):
    axs[1].text(agg_municipios_medios.index[-1]+.2, agg_municipios_medios.iloc[-1,i], agg_municipios_medios.columns[i],
                fontsize=15, fontweight='bold', color=axs[1].get_lines()[i].get_color())

for i in range(len(axs[2].get_lines())):
    axs[2].text(agg_municipios_pequenos.index[-1]+.2, agg_municipios_pequenos.iloc[-1,i], agg_municipios_pequenos.columns[i],
                fontsize=15, fontweight='bold', color=axs[2].get_lines()[i].get_color())

axs[0].axvline(2005, color='grey', linestyle='--')
axs[0].axvline(2013, color='grey', linestyle='--')
axs[0].axvline(2014, color='grey', linestyle='--')
axs[0].axvline(2016, color='grey', linestyle='--')

axs[1].axvline(2005, color='grey', linestyle='--')
axs[1].axvline(2013, color='grey', linestyle='--')
axs[1].axvline(2014, color='grey', linestyle='--')
axs[1].axvline(2016, color='grey', linestyle='--')

axs[2].axvline(2005, color='grey', linestyle='--')
axs[2].axvline(2013, color='grey', linestyle='--')
axs[2].axvline(2014, color='grey', linestyle='--')
axs[2].axvline(2016, color='grey', linestyle='--')

axs[0].text(0.275, 0.99, 'Mensalão', transform=axs[0].transAxes, rotation=-90,
            fontsize=15, color='dimgrey', verticalalignment='top')
axs[0].text(0.635, 0.99, 'Manifestações', transform=axs[0].transAxes, rotation=-90,
            fontsize=15, color='dimgrey', verticalalignment='top')
axs[0].text(0.685, 0.99, 'Lava Jato', transform=axs[0].transAxes, rotation=-90,
            fontsize=15, color='dimgrey', verticalalignment='top')
axs[0].text(0.775, 0.99, 'Impeachment', transform=axs[0].transAxes, rotation=-90,
            fontsize=15, color='dimgrey', verticalalignment='top')

plt.savefig('output/vereadores_ideologia.png', dpi=300, bbox_inches = 'tight')

Variação entre eleições

In [ ]:
fig, axs = plt.subplots(3,1, figsize = (20,20))

sns.set_style('whitegrid')
sns.despine()

c_grande = sns.catplot(data=agg_municipios_grandes_var, x='Ano', y='Variação', hue='Ideologia', kind='bar',
                      palette=colors, ax=axs[0])
c_medio = sns.catplot(data=agg_municipios_medios_var, x='Ano', y='Variação', hue='Ideologia', kind='bar'
                      , palette=colors, ax=axs[1])
c_pequeno = sns.catplot(data=agg_municipios_pequenos_var, x='Ano', y='Variação', hue='Ideologia', kind='bar'
                      , palette=colors, ax=axs[2])

fig.suptitle('Variação entre Vereadores Eleitos por Ideologia do Partido', fontsize=25, fontweight='bold',
             color='grey')

axs[0].set_title('Municípios Grandes', loc='left', fontsize=20, fontweight='bold', color='grey')
axs[1].set_title('Municípios Médios', loc='left', fontsize=20, fontweight='bold', color='grey')
axs[2].set_title('Municípios Pequenos', loc='left', fontsize=20, fontweight='bold', color='grey')

axs[0].set_xlabel('')
axs[1].set_xlabel('')
axs[2].set_xlabel('')

axs[0].set_ylabel('Variação', labelpad=2, fontsize=15, fontweight='bold', color='grey')
axs[1].set_ylabel('Variação', labelpad=2, fontsize=15, fontweight='bold', color='grey')
axs[2].set_ylabel('Variação', labelpad=2, fontsize=15, fontweight='bold', color='grey')

axs[0].set_xticklabels(labels=[ano_da_eleicao[i+1] for i in axs[0].get_xticks()], fontsize=15, color='grey')
axs[1].set_xticklabels(labels=[ano_da_eleicao[i+1] for i in axs[1].get_xticks()], fontsize=15, color='grey')
axs[2].set_xticklabels(labels=[ano_da_eleicao[i+1] for i in axs[2].get_xticks()], fontsize=15, color='grey')

vals = axs[0].get_yticks()
axs[0].set_yticklabels(['{:,.0%}'.format(x) for x in vals], fontsize=15, color='grey')
vals = axs[1].get_yticks()
axs[1].set_yticklabels(['{:,.0%}'.format(x) for x in vals], fontsize=15, color='grey')
vals = axs[2].get_yticks()
axs[2].set_yticklabels(['{:,.0%}'.format(x) for x in vals], fontsize=15, color='grey')

axs[0].spines['bottom'].set_color('grey')
axs[0].spines['left'].set_color('grey')
axs[1].spines['bottom'].set_color('grey')
axs[1].spines['left'].set_color('grey')
axs[2].spines['bottom'].set_color('grey')
axs[2].spines['left'].set_color('grey') 

l = axs[0].legend(loc='upper left', ncol=4, fontsize=15)
for text in l.get_texts():
    text.set_color('grey')
axs[1].get_legend().remove()
axs[2].get_legend().remove()

axs[0].text(1.0, 1.01, legend, transform=axs[0].transAxes, fontsize=12, ha='right', va='top', color='grey')

plt.close(2)
plt.close(3)
plt.close(4)
fig.tight_layout(pad=6)
plt.savefig('output/vereadores_ideologia_var.png', dpi=300, bbox_inches = 'tight')

# Mapa Brasil

In [ ]:
year=2020

In [ ]:
def calcular_forca_ideologia(esquerda, centro, direita):
    ideologia = ['Esquerda','Centro','Direita']
    ideologia_50_perc = ['Esquerda >50%','Centro >50%','Direita >50%']
    total = [esquerda, centro, direita]
    
    max_val = max(total)
    max_index = total.index(max_val)
    
    perc = max_val / sum(total)
    if perc > .5:
        return ideologia_50_perc[max_index]
    else:
        return ideologia[max_index]

In [ ]:
df_temp = df[df.ano_eleicao == 2020]
t = pd.pivot_table(df_temp[['codigo_municipio','ideologia']], index=['codigo_municipio'], columns=['ideologia'], aggfunc=len, fill_value=0)

df_temp = df_temp.drop_duplicates(subset='codigo_municipio',keep='first')
df_temp = df_temp[['codigo_municipio','nome_municipio','sigla_uf','GEOCOD_IBGE','COD_TSE','AJUSTE',
                'População','densidade','grupo_densidade','latitude','longitude']]
df_temp.set_index('codigo_municipio', inplace=True)

df_temp = pd.concat([df_temp, t],1, sort=False)

In [ ]:
df_temp['total_vereadores'] = df_temp['Esquerda'] + df_temp['Centro'] + df_temp['Direita']
df_temp['ideologia'] = df_temp.apply(lambda x : calcular_forca_ideologia(x['Esquerda'],x['Centro'],x['Direita']),axis=1)

df_temp.drop(['COD_TSE','AJUSTE','grupo_densidade','Centro','Direita','Esquerda','total_vereadores'],1, inplace=True)
df_temp = df_temp[['GEOCOD_IBGE','sigla_uf','nome_municipio','ideologia','latitude','longitude','População','densidade']]

In [ ]:
df_temp.to_excel('output/flourish_vereadores_mapa_populacao_'+str(year)+'.xlsx')

Municipios Grandes que cada ideologia possui maioria

In [ ]:
df_temp.sort_values('População', ascending=False, inplace=True)

In [ ]:
df_temp[(df_temp['População'] > 500000) & (df_temp.ideologia == 'Esquerda >50%') | (df_temp.ideologia == 'Esquerda')].head()

In [ ]:
df_temp[(df_temp['População'] > 500000) & (df_temp.ideologia == 'Centro >50%') | (df_temp.ideologia == 'Centro')].head()

In [ ]:
df_temp[(df_temp['População'] > 500000) & (df_temp.ideologia == 'Direita >50%') | (df_temp.ideologia == 'Direita')].head()

Mapa do Brasil com quantidade de vereadores eleitos com determinada ideologia do partido

In [ ]:
df_novo = df_temp.copy()
df_novo.reset_index(inplace=True)
df_novo['GEOCOD_IBGE'] = df_novo['GEOCOD_IBGE'].astype(str)

In [ ]:
df_mapa = gpd.read_file('input/geo/br_municipios/BRMUE250GC_SIR.shp')
df_mapa['NM_MUNICIP'] = df_mapa['NM_MUNICIP'].str.upper().apply(lambda x: unidecode(x))
df_mapa['GEOCOD_IBGE'] = df_mapa['CD_GEOCMU']
df_mapa.set_index('GEOCOD_IBGE', inplace=True)

df_mapa_novo = pd.merge(df_mapa, df_novo, on=['GEOCOD_IBGE'])

In [ ]:
df_mapa_uf = gpd.read_file('input/geo/br_unidades_da_federacao/BR_UF_2019.shp')

df_novo_uf = df_novo.groupby(['sigla_uf','ideologia'])['GEOCOD_IBGE'].count().unstack().reset_index()
df_novo_uf.fillna(0,inplace=True)
df_novo_uf['max'] = df_novo_uf[['Centro','Direita','Esquerda', 'Esquerda >50%','Centro >50%','Direita >50%']].idxmax(axis=1)

df_mapa_uf_novo = pd.merge(df_mapa_uf, df_novo_uf, left_on='SIGLA_UF', right_on='sigla_uf')
df_mapa_uf_novo.dropna(inplace=True)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,15))

p1 = df_mapa_novo.plot(column='ideologia', cmap=ListedColormap(['navajowhite','orange','lightskyblue','dodgerblue','lightcoral','red']), legend=True, linewidth=0.2, ax=axs[0], edgecolor='black')
p2 = df_mapa_uf_novo.plot(column='max', cmap=ListedColormap(['navajowhite','orange','red']), legend=True, linewidth=0.2, ax=axs[1], edgecolor='black')

p1.set_axis_off()
p2.set_axis_off()

fig.savefig('output/brasil_vereadors_municipios_'+str(year)+'.png', dpi=300, bbox_inches = 'tight')